In [1]:
# packages are loaded
import xarray as xr
import pint_xarray
import glob
import netCDF4 as nc4
import os
import pandas as pd
from   easymore import Easymore
import numpy as np
from   easymore import Utility as esmrut

In [2]:
# inputs
# Set the folder path where the remapped .nc file is located for MESH (it can be any remapped nc file)
path_to_save = '/home/shg096/scratch/test/SUMMA/'


In [3]:
attr = xr.open_dataset('/home/shg096/scratch/test/attr/attribute.nc')
forcing = xr.open_dataset('/home/shg096/scratch/test/SUMMA/SUMMA_forcing.nc')

# reorder based on the COMID order in the forcing
attr = esmrut.reorder(attr,
                      np.array(forcing['hru'].values),
                      mapping = {'var_id':'COMID','dim_id':'n'})

# prepare for the SUMMA attr file
attr ['hruId'] = attr ['COMID']
attr ['gruId'] = xr.DataArray(attr ['COMID'].values, dims=('gru'))
attr ['hru2gruId'] = attr ['COMID']
attr ['downHRUindex'] = xr.DataArray(np.zeros(len(attr['n'].values)), dims=('hru'))
attr ['elevation'] = attr['mean_elev']
attr ['HRUarea'] = attr['unitarea']
attr ['tan_slope'] = xr.DataArray(np.ones(len(attr['n'].values)), dims=('hru'))
attr ['contourLength'] = xr.DataArray(np.ones(len(attr['n'].values)), dims=('hru'))
attr ['slopeTypeIndex'] = xr.DataArray(np.ones(len(attr['n'].values)), dims=('hru'))
attr ['soilTypeIndex'] = xr.DataArray(attr ['soil_majority'].values.astype(int), dims=('hru'))
attr ['vegTypeIndex'] = xr.DataArray(attr ['LULC_majority'].values.astype(int), dims=('hru'))
attr ['mHeight'] = xr.DataArray(np.ones(len(attr['n'].values))*2, dims=('hru'))


# List of variables to keep
variables_to_keep = ['hruId', 'gruId', 'hru2gruId', 'downHRUindex',\
                     'elevation', 'HRUarea', 'tan_slope', 'contourLength',\
                     'slopeTypeIndex', 'vegTypeIndex', 'mHeight', 'longitude',\
                     'latitude', 'soilTypeIndex', 'latitude', 'longitude']
# Drop variables not in the list
attr = attr.drop_vars(set(attr.variables) - set(variables_to_keep))

# rename the n dimension to hru
attr = attr.rename({'n': 'hru'})

attr

if os.path.isfile(path_to_save+'SUMMA_attributes.nc'):
    os.remove(path_to_save+'SUMMA_attributes.nc')

attr.to_netcdf(path_to_save+'SUMMA_attributes.nc')

In [15]:
# Define dimensions
hru_size = len(attr['hruId'].values)
midSoil_size = 8
midToto_size = 8
ifcToto_size = 9
scalarv_size = 1

# Create a new xarray dataset
ds = xr.Dataset()

# Add dimensions to the dataset
ds['hru'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-'})
ds['midSoil'] = xr.DataArray(range(midSoil_size), dims=('midSoil'))
ds['midToto'] = xr.DataArray(range(midToto_size), dims=('midToto'))
ds['ifcToto'] = xr.DataArray(range(ifcToto_size), dims=('ifcToto'))
ds['scalarv'] = xr.DataArray(range(scalarv_size), dims=('scalarv'))

# Add variables to the dataset
ds['hruId'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-', 'long_name': 'Index of hydrological response unit (HRU)'})
ds['dt_init'] = xr.DataArray([[3600.0] * hru_size], dims=('scalarv', 'hru'))
ds['nSoil'] = xr.DataArray([[8] * hru_size], dims=('scalarv', 'hru'))
ds['nSnow'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyIce'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyLiq'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSnowDepth'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSWE'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSfcMeltPond'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarAquiferStorage'] = xr.DataArray([[1.0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSnowAlbedo'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanairTemp'] = xr.DataArray([[283.16] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyTemp'] = xr.DataArray([[283.16] * hru_size], dims=('scalarv', 'hru'))
ds['mLayerTemp'] = xr.DataArray([[283.16] * hru_size ] * midToto_size , dims=('midToto', 'hru'))
ds['mLayerVolFracIce'] = xr.DataArray([[0] * hru_size] * midToto_size, dims=('midToto', 'hru'))
ds['mLayerVolFracLiq'] = xr.DataArray([[0.2] * hru_size] * midToto_size, dims=('midToto', 'hru'))
ds['mLayerMatricHead'] = xr.DataArray([[-1] * hru_size] * midToto_size, dims=('midSoil', 'hru'))
# ds['iLayerHeight'] = xr.DataArray([[0.000,0.025,0.100,0.250,0.500,1.000,1.500,2.500,4.000]] * hru_size , dims=( 'hru', 'ifcToto',))
# ds['mLayerDepth'] = xr.DataArray([[0.025,0.075,0.150,0.250,0.500,0.500,1.000,1.500]] * hru_size, dims=( 'hru', 'midToto',))
ds['iLayerHeight'] = xr.DataArray(np.transpose([[0.000,0.025,0.100,0.250,0.500,1.000,1.500,2.500,4.000]] * hru_size) , dims=('ifcToto', 'hru'))
ds['mLayerDepth'] = xr.DataArray(np.transpose([[0.025,0.075,0.150,0.250,0.500,0.500,1.000,1.500]] * hru_size), dims=('midToto',  'hru'))


if os.path.isfile(path_to_save+'SUMMA_coldState.nc'):
    os.remove(path_to_save+'SUMMA_coldState.nc')

ds.to_netcdf(path_to_save+'SUMMA_coldState.nc')



In [16]:
# Define dimensions
hru_size = len(attr['hruId'].values)

# Create a new xarray dataset
ds = xr.Dataset()

# Add dimensions to the dataset
ds['hru'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-'})

# Add variables to the dataset
ds['hruId'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-', 'long_name': 'Index of hydrological response unit (HRU)'})

if os.path.isfile(path_to_save+'SUMMA_trialParams.nc'):
    os.remove(path_to_save+'SUMMA_trialParams.nc')

ds.to_netcdf(path_to_save+'SUMMA_trialParams.nc')